In [76]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wavfile
import scipy.fft as sp_fft
import scipy.signal as sp_signal
import os


##### Ensure the output directory exists


In [ ]:
output_dir = "newaudio"
os.makedirs(output_dir, exist_ok=True)


##### read voice file and return amplitude and frequency


In [ ]:
def read_voice(path):
    rate, data = wavfile.read(path)
    Amplitude = sp_fft.rfft(data)
    Frequency = sp_fft.rfftfreq(len(data), 1 / rate)
    return rate, data, Amplitude, Frequency


##### change voice speed by changing the rate


In [ ]:
def change_voice_speed(data, rate, speed_factor):
    speed_factor = np.sqrt(speed_factor)
    new_rate = int(rate * speed_factor)
    new_length = int(len(data) / speed_factor)
    new_data = sp_signal.resample(data, new_length)
    return new_data, new_rate


##### low pass filter to remove noise from voice


In [ ]:
def low_pass_filter(Frequency, Amplitude, cutoff):
    filtered_amplitude = Amplitude.copy()
    filtered_amplitude[Frequency > cutoff] = 0
    filtered_amplitude[np.abs(Amplitude) > 1e8] = 0
    return filtered_amplitude


##### reverse the voice to play it backwards


In [ ]:
def reverse_voice(data, rate):
    return data[::-1], rate  # reverse the data


##### mix voices


In [ ]:
def mix_voices(Datas, Rates):
    min_rate = min(Rates)
    resampled_datas = [sp_signal.resample(data, int(len(data) * min_rate / rate)) for data, rate in zip(Datas, Rates)]
    max_length = max(len(data) for data in resampled_datas)
    normalized_datas = [np.pad(data, (0, max_length - len(data)), 'constant') for data in resampled_datas]
    mixed_data = np.sum(normalized_datas, axis=0)
    mixed_data = mixed_data / len(Datas)  # Normalize to avoid clipping
    return mixed_data.astype(np.int16), min_rate


##### write the voice to a file


In [ ]:
def write_voice(data, rate, path):
    wavfile.write(path, rate, data.astype(np.int16))
    return


##### plot and save the voice


In [ ]:
def plot_voice(Frequency, Amplitude, title="Frequency Domain Representation", filename=None):
    plt.plot(Frequency, np.abs(Amplitude))
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Amplitude")
    plt.title(title)
    if filename:
        plt.savefig(filename)
    plt.close()


##### plot and save the spectrogram


In [ ]:
def show_spectrogram(data, rate, title="Spectrogram", filename=None):
    plt.specgram(data, Fs=rate)
    plt.title(title)
    if filename:
        plt.savefig(filename)
    plt.close()


##### plot and save the waveform


In [ ]:
def plot_waveform(data, rate, title="Waveform", filename=None):
    time = np.arange(len(data)) / rate
    plt.plot(time, data)
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.title(title)
    if filename:
        plt.savefig(filename)
    plt.close()


#### Clean the voice ####


In [ ]:
# Original voice path
input_path = "audio/potc.wav"

# Cleaned voice path
cleanpotc = os.path.join(output_dir, "cleanpotc.wav")

# Read the voice file
rate, data, Amplitude, Frequency = read_voice(input_path)

# Save original voice plots
plot_voice(Frequency, Amplitude, title="Original Frequency Domain Representation", filename=os.path.join(output_dir, "original_Amplitude.png"))
plot_waveform(data, rate, title="Original Waveform", filename=os.path.join(output_dir, "original_Data.png"))
show_spectrogram(data, rate, title="Original Spectrogram", filename=os.path.join(output_dir, "original_Spectrogram.png"))

# cutoff frequency for low pass filter
cutoff_frequency = 4000
# Apply low pass filter
filtered_Amplitude = low_pass_filter(Frequency, Amplitude, cutoff_frequency)

# Save filtered voice plots
plot_voice(Frequency, filtered_Amplitude, title="Filtered Frequency Domain Representation", filename=os.path.join(output_dir, "cleanpotc_Amplitude.png"))

# Inverse FFT to get back to time domain
filtered_data = sp_fft.irfft(filtered_Amplitude)

# Write the filtered voice to a new file
write_voice(filtered_data.astype(np.int16), rate, cleanpotc)

# Save cleaned voice plots
plot_waveform(filtered_data, rate, title="Filtered Waveform", filename=os.path.join(output_dir, "cleanpotc_Data.png"))
show_spectrogram(filtered_data, rate, title="Filtered Spectrogram", filename=os.path.join(output_dir, "cleanpotc_Spectrogram.png"))


### Change the speed of the voice ####


In [ ]:

# Consider the cleaned voice as the input voice
input_path = cleanpotc

# Speed changed voice path
fastpotc = os.path.join(output_dir, "fastpotc.wav") # 2x speed
slowpotc = os.path.join(output_dir, "slowpotc.wav") # 0.5x speed

# Read the voice file
rate, data, _, _ = read_voice(input_path)

# Change the speed of the voice
fast_data, fast_rate = change_voice_speed(data, rate, 2)
slow_data, slow_rate = change_voice_speed(data, rate, 0.5)

# Write the fast voice to a new file
write_voice(fast_data, fast_rate, fastpotc)

# Write the slow voice to a new file
write_voice(slow_data, slow_rate, slowpotc)

# Save fast voice plots
plot_waveform(fast_data, fast_rate, title="Fast Waveform", filename=os.path.join(output_dir, "fastpotc_Data.png"))
show_spectrogram(fast_data, fast_rate, title="Fast Spectrogram", filename=os.path.join(output_dir, "fastpotc_Spectrogram.png"))

# Save slow voice plots
plot_waveform(slow_data, slow_rate, title="Slow Waveform", filename=os.path.join(output_dir, "slowpotc_Data.png"))
show_spectrogram(slow_data, slow_rate, title="Slow Spectrogram", filename=os.path.join(output_dir, "slowpotc_Spectrogram.png"))


#### Reverse the voice ####


In [ ]:
# Consider the cleaned voice as the input voice
input_path = cleanpotc

# Reversed voice path
revpotc = os.path.join(output_dir, "revpotc.wav")

# Read the voice file
rate, data, _, _ = read_voice(input_path)

# Reverse the voice
reversed_data, reversed_rate = reverse_voice(data, rate)

# Write the reversed voice to a new file
write_voice(reversed_data, reversed_rate, revpotc)

# Save reversed voice plots
plot_waveform(reversed_data, rate, title="Reversed Waveform", filename=os.path.join(output_dir, "revpotc_Data.png"))
show_spectrogram(reversed_data, rate, title="Reversed Spectrogram", filename=os.path.join(output_dir, "revpotc_Spectrogram.png"))


#### Mix voices ####


In [ ]:
# inputs are all the voices in the previous steps
input_paths = [cleanpotc, fastpotc, slowpotc, revpotc]

# Mixed voice path
mixpotc = os.path.join(output_dir, "mixpotc.wav")

DataArray = []
RateArray = []

for path in input_paths:
    rate, data, _, _ = read_voice(path)
    DataArray.append(data)
    RateArray.append(rate)

# Mix the voices
mixed_data, mixed_rate = mix_voices(DataArray, RateArray)

# Write the mixed voice to a new file
write_voice(mixed_data, mixed_rate, mixpotc)

# Save mixed voice plots
plot_waveform(mixed_data, mixed_rate, title="Mixed Waveform", filename=os.path.join(output_dir, "mixpotc_Data.png"))
show_spectrogram(mixed_data, mixed_rate, title="Mixed Spectrogram", filename=os.path.join(output_dir, "mixpotc_Spectrogram.png"))

mixed_Amplitude = sp_fft.rfft(mixed_data)
mixed_Frequency = sp_fft.rfftfreq(len(mixed_data), 1 / mixed_rate)
plot_voice(mixed_Frequency, mixed_Amplitude, title="Mixed Frequency Domain Representation", filename=os.path.join(output_dir, "mixpotc_Amplitude.png"))